## Wykład 11 - `Retrofit2`, `URL`, Adnotacje, Procesor Adnotacji

`Retrofit2` jest biblioteką do komunikacji z serwerami **HTTP**, która umożliwia łatwe tworzenie interfejsów **API**. Dzięki niej możesz skonfigurować adres **URL** serwera, a następnie **utworzyć interfejs, który opisuje pożądane zapytania HTTP** (np. GET, POST, PUT itp.). Retrofit **automatycznie mapuje** odpowiedzi z serwera na obiekty w twojej aplikacji.

Retrofit jest oparty o język Java i działa z Androidem, ale może być również używany z **innymi platformami Javy**. `Retrofit2` jest następną wersją `Retrofit`, która zawiera kilka nowych funkcji i usprawnień w stosunku do pierwszej wersji.

`Retrofit2` pozwala na:

- Umożliwia łatwe tworzenie interfejsów API
- Pozwala na dynamiczne zmienianie adresu URL serwera na podstawie danych z aplikacji
- Automatycznie mapuje odpowiedzi z serwera na obiekty w aplikacji za pomocą konwerterów
- Pozwala na wysyłanie zapytań z różnymi typami ciała (np. JSON, Form-data)
- Obsługa asynchronicznego przetwarzania zapytań
- Wsparcie dla wielu konwerterów (np. GSON, Moshi, Jackson)
- Obsługa adnotacji do oznaczania parametrów zapytania, nagłówków, ciała zapytania

lista konwerterów, które są dostępne w Retroficie:

- **Gson**: Ten konwerter używa biblioteki **Gson** do serializacji i deserializacji obiektów **Java** na i z formatu **JSON**.

- **Jackson**: Ten konwerter używa biblioteki Jackson do serializacji i deserializacji obiektów **Java** na i z formatu **JSON**.

- **Moshi**: Ten konwerter używa biblioteki Moshi do serializacji i deserializacji obiektów **Java** na i z formatu **JSON**.

- **Protobuf**: Ten konwerter używa biblioteki **Protobuf** do serializacji i deserializacji obiektów Java na i z formatu **Protobuf**.

- **Wire**: konwerter używa biblioteki Wire do serializacji i deserializacji obiektów **Java** na i z formatu **Protobuf**.

- **Simple XML**: konwerter używa biblioteki `Simple XML` do serializacji i deserializacji obiektów **Java** na i z formatu **XML**.

- **Scalars** (określone typy jak `String`, `Number`, `Boolean`): konwerter pozwala na przetwarzanie odpowiedzi HTTP na typy podstawowe, takie jak `String`, `Number`, `Boolean`.

- **FormUrlEncoded** i **Multipart**: konwerter pozwala na przesyłanie danych za pomocą metod `POST` i `PUT` w formacie `x-www-form-urlencoded` i `multipart/form-data`.

Retrofit pozwala na dodanie kilku konwerterów i wybieranie odpowiedniego na podstawie typu odpowiedzi.

**JSON** (JavaScript Object Notation) jest formatem przesyłania danych, który jest lekki i łatwy do przetwarzania dla maszyn. Jest on bardzo podobny do składni obiektów **JavaScript**, dlatego jego nazwa zawiera "JavaScript".

**JSON** jest formatem tekstowym, który składa się z pary klucz-wartość. Klucze są ciągami tekstowymi, a wartości mogą być różnymi typami danych, takimi jak liczby, ciągi znaków, obiekty i tablice.

Przykład:
```verbatim
{
  "name": "John",
  "age": 30,
  "address": {
    "street": "Main St",
    "city": "New York"
  },
  "phones": [
    "+123456789",
    "+987654321"
  ]
}
```

**GSON** jest biblioteką, która pozwala na konwersję danych między różnymi formatami (np. **JSON**, **Java**) w języku **Java**. **GSON** jest projektem open-source i jest rozwijany przez Google.

**GSON** pozwala na konwersję obiektów Java na format JSON oraz odwrotnie. Dzięki temu, że jest ona w pełni zintegrowana z językiem Java, można ją łatwo używać do parsowania odpowiedzi z serwera do obiektów w aplikacji.

**GSON** umożliwia:

- automatyczne mapowanie odpowiedzi **JSON** na obiekty Java
- konfigurację, aby ignorować pola, które nie powinny być mapowane
- mapowanie pola, które ma inną nazwę niż nazwa pola w klasie Java

**GSON** jest bardzo popularną biblioteką w branży mobilnej i jest często używana w połączeniu z biblioteką **Retrofit2**, aby umożliwić automatyczne mapowanie odpowiedzi z serwera na obiekty w aplikacji.

Do aplikacji musimy dodać odpowiednie zależności - przechodzimy do pliku `build.gradle.kts`.

Dodajemy dwie zależności (można wyszukać wciskając **Alt + Insert**)

In [ ]:
dependencies {
    implementation("com.squareup.retrofit2:retrofit:2.9.0")
    implementation("com.squareup.retrofit2:converter-gson:2.9.0")
    ...
}

Po czym musimy wykonać **synchronizację** (**load Gradle changes**) projektu - **Ctrl + Shift + O**

W tym przykładzie wykorzystamy **[JSONPlaceholder](https://jsonplaceholder.typicode.com/)**, który jest API przeznaczonym do testowania. Mamy dostępnych kilka endpointów

- posts
- comments
- albums
- photos
- todos
- users


W tym przykładzie wybierzemy pierwszy (`posts`), rozpoczniemy komunikację z tym serwerem oraz **asynchronicznie** wykonamy operację `GET` - czyli pobierzemy wszystkie posty i je wyświetlimy.

Posty znajdziemy w formacie **JSON**

In [ ]:
{
    "userId": 1,
    "id": 1,
    "title": "sunt aut ...",
    "body": "quia et ..."
},

W pierwszym kroku musimy stworzyć nasz **model danych** odpowiadający strukturze obecnej na serwerze. Więc tworzymy klasę `Post` i dodajemy pola `userId`, `id`, `title` oraz `body`. Jeżeli chcemy wykorzystać inną nazwę musimy użyć adnotacji `@SerializedName` - tutaj zmienimy nazwę `body` na `content`. W argumencie `@SerializedName` podajemy nazwę którą chcemy zmienić jako `String` - czyli ta nazwa musi odpowiadać nazwie obecnej w formacie dostępnym na serwerze.

In [ ]:
// kotlin
data class Post (
    val userId: Int,
    val id: Int,
    val title: String,

    @SerializedName("body")
    val content: String
)

In [ ]:
// java
public class Post {
    private int userId;
    private int id;
    private String title;

    @SerializedName("body")
    String content;

    public int getUserId() {
        return userId;
    }

    public int getId() {
        return id;
    }

    public String getTitle() {
        return title;
    }

    public String getContent() {
        return content;
    }
}

Teraz musimy stworzyć interfejs w którym zdefiniujemy metodę służącą zwróceniu danych z serwera.



Zwracanym obiektem jest `Call` zawierający listę wszystkich postów. Oprócz danych w tym obiekcie znajdują się również obiekty `Response` oraz `Request` zawierające informacje o samym połączeniu. W klasie `Call` dostępna jest metoda `enqueue` pozwalająca na **asynchroniczne** wysłanie żądania oraz powiadomienia zwrotnego, lub błędu - innymi słowy `Call` hermetyzuje pojedynczy `Request` oraz pojedynczy `Response`.

Musimy wykorzystać adnotację `@GET` aby poinformować `Retrofit` co dokładnie ta metoda ma robić. Dzięki temu `Retrofit` będzie w stanie wygenerować odpowiedni kod.

In [ ]:
// kotlin
import retrofit2.Call
import retrofit2.http.GET

interface PlaceholderApi {
    @GET("posts")
    fun posts(): Call<List<Post>>
}

In [ ]:
// java
public interface PlaceholderApi {
    @GET("posts")
    Call<List<Post>> getPosts();
}

Ciało metody `getPosts` (`posts` w Kotlinie) zostanie wygenerowane automatycznie przez **Annotation Proccessor**. Informację o tym jaki kod ma zostać wygenerowany dostarczamy za pomocą adnotacji `@GET`

**Annotation Processor** to narzędzie, które pozwala na przetwarzanie **adnotacji** (annotations) w języku Java/Kotlin podczas kompilacji. **Adnotacje** to specjalne znaczniki, które mogą być dodawane do elementów języka (np. klasy, metody, pola) i które służą do opisania pewnych cech tych elementów. **Annotation Processor** pozwala na automatyczne wykonywanie pewnych operacji na elementach oznaczonych adnotacjami podczas kompilacji.

**Annotation Processor** umożliwia:

- generowanie kodu na podstawie adnotacji
- walidację elementów oznaczonych adnotacjami podczas kompilacji
- generowanie plików dodatkowych, takich jak pliki konfiguracyjne lub pliki XML

**Annotation Processor** jest często używany w różnych bibliotekach i frameworkach Java, takich jak Android, Spring, Hibernate, i pozwala na automatyzację procesu tworzenia kodu, zwiększając jego jakość oraz zwiększając produktywność developerów.

Następnie w metodzie `main` musimy utworzyć obiekt `Retrofit`

In [ ]:
// kotlin
val retrofit = Retrofit.Builder()
    .baseUrl("https://jsonplaceholder.typicode.com/")
    .addConverterFactory(GsonConverterFactory.create())
    .build()

In [ ]:
// java
Retrofit retrofit = new Retrofit.Builder()
        .baseUrl("https://jsonplaceholder.typicode.com/")
        .addConverterFactory(GsonConverterFactory.create())
        .build();

Posiadając instancję `Retrofit` możemy utworzyć instancję obiektu o typie interfejsu `PlaceholderApi`.

In [ ]:
// kotlin
val api = retrofit.create(PlaceholderApi::class.java)

In [ ]:
// java
PlaceholderApi api = retrofit.create(PlaceholderApi.class);

Chcemy wykonać operację sieciową **asynchronicznie** - tutaj `Retrofit` dostarcza odpowiednią metodę (`enqueue`), której parametrem jest obiekt o typie interfejsu `Callback`.

Na tym obiekcie możemy również wykonać `execute` - jest to wykonanie **synchroniczne**.

Wpierw utwórzmy obiekt `Call`

In [ ]:
// kotlin
val call: Call<List<Post>> = api.posts()

In [ ]:
// java
Call<List<Post>> call = api.getPosts();

In [ ]:
// kotlin
call.enqueue(object : Callback<List<Post>> {
    override fun onResponse(
        call: Call<List<Post>?>, 
        response: Response<List<Post>?>) 
    {}

    override fun onFailure(call: Call<List<Post>?>, t: Throwable) 
    {}
})

In [ ]:
// java
call.enqueue(new Callback<List<Post>>() {
    @Override
    public void onResponse(
        @NonNull Call<List<Post>> call,
        @NonNull Response<List<Post>> response) {
    }

    @Override
    public void onFailure(
        @NonNull Call<List<Post>> call, 
        @NonNull Throwable t) {
    }
}

Obiekt `Call` jest podstawowym obiektem używanym przez bibliotekę `Retrofit` do reprezentowania połączenia z serwerem. Obiekt `Call` jest używany do tworzenia i wykonywania zapytań **HTTP**.

posiada metody pozwalające na:

- Ustawienie nagłówków, parametrów zapytania, ciała zapytania
- Ustawienie konwertera danych
- Określenie, czy połączenie ma być wykonywane asynchronicznie czy synchronicznie
- Określenie, czy połączenie ma być cache'owane

Obiekt `Response` jest obiektem zwracanym przez metodę `execute()` lub `enqueue()` z obiektu `Call` z biblioteki `Retrofit`. Zawiera on informacje dotyczące odpowiedzi z serwera, takie jak status **HTTP, nagłówki, ciało odpowiedzi**.

Obiekt `Response` posiada następujące metody:

- isSuccessful(): zwraca `true`, jeśli status HTTP odpowiedzi jest pomiędzy 200 a 300
- code(): zwraca status HTTP odpowiedzi
- headers(): zwraca nagłówki odpowiedzi
- body(): zwraca ciało odpowiedzi
- errorBody(): zwraca ciało odpowiedzi, jeśli status HTTP jest poza zakresem od 200 do 300

Musimy zaimplementować dwie metody
- `onResponse` - wykonywana przy sukcesie komunikacji z serwerem - co oznacza samą komunikację a nie powodzenie samej operacji (przykładowo możemy dostać znany kod 404 przy próbie dostępu do danych które nie istnieją na serwerze)
- `onFailure` - wykonywana przy braku komunikacji z serwerem

W pierwszym kroku implementacji tej metody sprawdzamy czy odpowiedź jest poprawna

In [ ]:
if (response.isSuccessful) {}

Czyli kod który otrzymujemy mieści się w zakresie 200 - 300 - więcej informacji o kodach [tutaj](https://developer.mozilla.org/en-US/docs/Web/HTTP/Status). 

Następnie musimy rozpakować dane - są one przechowywane w polu `body` obiektu `Response`. Oprócz tego `Response` posiada jeszcze pole typu `okhttp3.Response` zawierające kod odpowiedzi, rodzaj wykorzystanego protokołu (`HTTP 1.1`), oraz kilka innych informacji - w tym przykładzie są one nie istotne.

In [ ]:
// kotlin
val posts = response.body()

In [ ]:
// java
List<Post> posts = response.body();

In [ ]:
// kotlin
call.enqueue(object : Callback<List<Post>> {
    override fun onResponse(
        call: Call<List<Post>?>, 
        response: Response<List<Post>?>) 
    {
        if (response.isSuccessful) {
            
            val posts = response.body()

            posts?.forEach {
                val content = StringBuilder()
                content.append("id: ")
                    .append(it.id)
                    .append("\n")
                    .append("UserId: ")
                    .append(it.userId)
                    .append("\n")
                    .append("title: ")
                    .append(it.title)
                    .append("\n")
                    .append("body: ")
                    .append(it.content)
                    .append("\n\n")
                println(content)
            }
        } else {
            println("fail")
        }
    }

    override fun onFailure(
        call: Call<List<Post>?>, 
        t: Throwable) {
        println("error")
    }
})

In [ ]:
// java
call.enqueue(new Callback<List<Post>>() {
    @Override
    public void onResponse(
            Call<List<Post>> call,
            Response<List<Post>> response) 
    {
        if (response.isSuccessful()) {
            List<Post> posts = response.body();
            if (posts != null) {
                posts.forEach(post -> {
                    StringBuilder content = new StringBuilder();
                    content
                            .append("id: ")
                            .append(post.getId())
                            .append("\n")
                            .append("UserId: ")
                            .append(post.getUserId())
                            .append("\n")
                            .append("title: ")
                            .append(post.getTitle())
                            .append("\n")
                            .append("body: ")
                            .append(post.getContent())
                            .append("\n\n");
                    System.out.println(content);
                });
            }
        } else {
            System.out.println("fail");
        }
    }

    @Override
    public void onFailure(
            Call<List<Post>> call,
            Throwable t) 
    {
        System.out.println("error");
    }
});

Zazwyczaj potrzebujemy tylko jakiś podzbiór wszystkich dostępnych danych. W tym celu posługujemy się odpowiednimi parametrami w adresie URL. W sekcji **Routes** mamy podane różne rodzaje metod HTTP

- GET /posts
- GET /posts/1
- GET /posts/1/comments
- GET /comments?postId=1

## Adnotacje

### **@Path**

Jeżeli chcemy napisać metodę przyjmującą jako parametr `id` posta, musimy wykorzystać odpowiednią adnotację.

In [ ]:
@GET("posts/{id}/comments")
fun getComments(@Path("id") postId: Int): Call<List<Comment>>

Dzięki zastosowaniu adnotacji `@Path` przekazujemy informację o zastosowaniu parametru metody jako składowej adresu `URL`. Tutaj musimy zwrócić uwagę na parametr `@Path` `"id"` - musi on być zgodny z częścią parametru adnotacji `@GET`, który podajemy w nawiasach `{id}`. Teraz wywołanie metody wygląda następująco

In [ ]:
val call = service.getComments(3)

### **@Query**

Mamy również drugi sposób przekazania argumentu i uzyskania tej samej informacji - */comments?postId=1*. Dostaniemy dokładnie ta samą informację - wszystkie komentarza pod postem o zadanym `id`. Tym razem musimy wykorzystać adnotację `@Query` - zapytanie jest rozpoczynane symbolem `?`.

Dodajmy metodę `getCommentsFromQuery`

In [ ]:
@GET("comments")
fun getCommentsFromQuery(@Query("postId") postId: Int)
    : Call<List<Comment>>

Tutaj parametr adnotacji `@Query` musi odpowiadać nazwie parametru (lub metody) obecnej w zapytaniu - tutaj będzie to */comments?postId=1*. Pozostałe elementy zapytania (znak rozpoczynający zapytanie oraz symbol `=`) zostanie dodany automatycznie. Metodę wywołujemy tak samo jak poprzednią

In [ ]:
val call = service.getCommentsFromQuery(3)

Możemy również podać wiele parametrów - ich nazwy i wszystkie możliwości znadziemy w dokumentacji na stronie z  której korzystamy.

Chcemy uzyskać listę wszystkich komentarzy pod postem o zadanych `id`, posortowane po identyfikatorze malejąco. `url` będzie wyglądał następująco
- */comments?postId=1&_sort=id&_order=desc* - znak `&` rozdziela parametry

Napiszmy odpowiednią metodę

In [ ]:
@GET("comments")
fun getSortedComments(
    @Query("postId") postId: Int,
    @Query("_sort") sort: String,
    @Query("_order") order: String
): Call<List<Comment>>

In [ ]:
val call = service.getSortedComments(2, "id", "desc")

Zwróćmy uwagę że parametr sortowania podajemy jako `String`.

Jeżeli chcemy dostać wszystkie komentarze posortowane malejąco po `id`, możemy przekazać jako parametr `posrId` wartość `null`

In [ ]:
val call = service.getSortedComments(null, "id", "desc")

Jeżeli chcemy dostać listę komentarzy z kilku postów, możemy zadeklarować metodę przyjmującą tablicę (lub listę) identyfikatorów

In [ ]:
@GET("comments")
fun getSortedCommentsFromPosts(
    @Query("postId") postsId: List<Int>,
    @Query("_sort") sort: String,
    @Query("_order") order: String
): Call<List<Comment>>

In [ ]:
val call = service.getSortedCommentsFromPosts(
    listOf(1, 3, 6, 7), "id", "desc");

### **@Url**

Jeżeli adres jest skomplikowany z większą ilością parametrów, możemy chieć przekazać sam `url` jako parametr funkcji.

In [ ]:
@GET
fun getComments(
    @Url url: String
): Call<List<Comment>>

In [ ]:
val call = service.getComments("comments?postId=3")

val call = service.getComments(
    "https://jsonplaceholder.typicode.com/comments?postId=3")